# Dev 03 - Spectral Processing

Using the `toshi` module to read in data and process for spectra

In [1]:
import os
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

plt.style.use('./darkmode.style')

In [2]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

Data read in 2.00 s


In [3]:
naz = len(ray_pulses)
ngate = ray_pulses[0][0].ngate_long_hi
# ngate = ray_pulses[0][0].ngate_short_hi
z = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)
e = np.zeros((naz), dtype=np.single)
a = np.zeros((naz), dtype=np.single)
r = np.zeros((ngate), dtype=np.single)

for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
    pp = p[1:, :] * np.conj(p[:-1, :])
    v[k, :] = np.angle(np.sum(pp, axis=0))
    z[k, :] = np.sum(np.abs(p), axis=0)

# Sweep operations
z = 20 * np.log10(z) - 60
# v[z < -10] = np.nan

In [ ]:
w, h = 800, 450
dpi = 72
fig = plt.figure(figsize=[x / dpi for x in (w, h)], dpi=dpi)


In [ ]:
# import struct

# pri_struct = struct.Struct('I'*32)
# n = pri_struct.unpack_from(ray_pulses[0][1].raw_pri)
# for k, x in enumerate(n):
#     print('{:2d}: {:08X}'.format(k, x))

In [70]:
# Radar coordinate
# Approximate location from Google Maps: 39.7294019,140.0440751
lat, lon = 39.7294019, 140.0440751
re = 6371
lam = np.deg2rad(lon)
phi = np.deg2rad(lat)

x = re * np.cos(phi) * np.cos(lam)
y = re * np.cos(phi) * np.sin(lam)
z = re * np.sin(phi)

# Fly to the top of the radar so that:
# x-axis is due east
# y-axis is due norrth
# z-axis is straight up from the radar

# Equivalent:
# Rotate the earth about the z-axis by -(90 + lon), then
# Rotate the earth about the x-axis by -(90 - lat)
za = -(0.5 * np.pi + lam)
rz = np.array([[np.cos(za), -np.sin(za), 0], [np.sin(za), np.cos(za), 0], [0, 0, 1]])

xa = -(0.5 * np.pi - phi)
rx = np.array([[1, 0, 0], [0, np.cos(xa), -np.sin(xa)], [0, np.sin(xa), np.cos(xa)]])

p = np.array([x, y, z])

qq = np.matmul(rz, p)
q = np.matmul(rx, np.matmul(rz, p))

In [129]:
def geo2cart(lat, lon, olat, olon):
    # Convention:
    # Longitude = lam = angle from x-axis
    # Latitude = phi = angle from xy-plane
    re = 6371e3
    lam = np.deg2rad(lon)
    phi = np.deg2rad(lat)
    # Step 1 - rotate the earth about the z-axis by -(90 + lon)
    az = np.deg2rad(-(90 + olon))
    rz = np.array([[np.cos(az), -np.sin(az), 0], [np.sin(az), np.cos(az), 0], [0, 0, 1]])
    # Step 2 - rotate the earth about the x-axis by -(90 - lat)
    ax = np.deg2rad(-(90 - olat))
    rx = np.array([[1, 0, 0], [0, np.cos(ax), -np.sin(ax)], [0, np.sin(ax), np.cos(ax)]])
    # Matrix multiplication form
    p = np.array([
        np.cos(phi) * np.cos(lam),
        np.cos(phi) * np.sin(lam),
        np.sin(phi)
    ])
    q = re * np.matmul(rx, np.matmul(rz, p))
    return q[0], q[1], q[2] - re

In [136]:
x, y, z = geo2cart(lat + 0.01, lon, olat=lat, olon=lon)
# x, y, z = geo2cart(lat, lon, lat, lon)

In [137]:
print(x, y, z)

3.536615444943436e-10 1111.9492607993116 -0.09703587926924229
